In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()



In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-06 15:08:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-08-06 15:08:39 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark.sql.functions import lit
#from pyspark import SparkFiles

#url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
#spark.sparkContext.addFile(url)

#sc = spark.sparkContext

#sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "ACCESSKEY")
#sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "SecretKey")
#sc._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
#sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")

# s3a://amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz

lawngarden_df = spark.read.option("header", "true").csv("gdrive/My Drive/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz", inferSchema=True, sep="\t")

#lawngarden_df = spark.read.format('csv').options(header='true', inferSchema='true', delimiter='\t').load('s3a://amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz')

lawngarden_df = lawngarden_df.withColumn('category',lit('LawnGarden'))

lawngarden_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-----------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
lawngarden_df.count()

2557288

In [7]:
customer_df = lawngarden_df.select(["customer_id","category","review_id"])\
  .groupby("customer_id","category")\
  .agg({"review_id":"count"})

customer_df = customer_df.withColumnRenamed("count(review_id)","customer_count")
customer_df.show(truncate=False)

+-----------+----------+--------------+
|customer_id|category  |customer_count|
+-----------+----------+--------------+
|52708009   |LawnGarden|1             |
|32761415   |LawnGarden|1             |
|140799     |LawnGarden|1             |
|37522403   |LawnGarden|1             |
|33869860   |LawnGarden|1             |
|35444302   |LawnGarden|3             |
|44332738   |LawnGarden|1             |
|44240776   |LawnGarden|3             |
|52506746   |LawnGarden|1             |
|13516043   |LawnGarden|1             |
|11773385   |LawnGarden|1             |
|33912036   |LawnGarden|1             |
|36971660   |LawnGarden|3             |
|18241774   |LawnGarden|1             |
|17141249   |LawnGarden|1             |
|9172326    |LawnGarden|1             |
|50499761   |LawnGarden|1             |
|30962243   |LawnGarden|7             |
|33341678   |LawnGarden|1             |
|28310867   |LawnGarden|2             |
+-----------+----------+--------------+
only showing top 20 rows



In [8]:
product_df = lawngarden_df.select('product_id','product_title').distinct()
product_df.show(truncate=False)

+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                  |
+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|B0081SBO4Y|Key Pair Lawn With Indak Key Switch For Mtd Kohler Briggs Husqvarna Roper Ayp                                                                                                                  |
|B00EGFKOZ6|Swiftly Done Bright Solar Power Outdoor LED Light No Tools Required Peel and Stick Motion Activated                                                                     

In [9]:
from pyspark.sql.types import DateType

review_df = lawngarden_df.select('review_id','customer_id','product_id','product_parent','category','review_date')
review_df=review_df.withColumn('review_date',review_df['review_date'].cast(DateType()))

review_df.show()

+--------------+-----------+----------+--------------+----------+-----------+
|     review_id|customer_id|product_id|product_parent|  category|review_date|
+--------------+-----------+----------+--------------+----------+-----------+
| RED72VWWCOS7S|   32787517|B008HDQYLQ|     348668413|LawnGarden| 2015-08-31|
| RZHWQ208LTEPV|   16374060|B005OBZBD6|     264704759|LawnGarden| 2015-08-31|
|R37LBC3XAVLYOO|    9984817|B00RQL8U2G|      95173602|LawnGarden| 2015-08-31|
|R3L7XJMA0MVJWC|   12635190|B0081SBO4Y|     835659279|LawnGarden| 2015-08-31|
|R2I2GHSI7T1UBN|   43905102|B008E6OK3U|     539243347|LawnGarden| 2015-08-31|
|R2GFFKHK4I6VMX|   52596997|B00W6NTULY|     337446474|LawnGarden| 2015-08-31|
|R1R0UDX2XAN1S4|   43871104|B00GXUMYKA|     468857193|LawnGarden| 2015-08-31|
|R22C8FMBSTFRY8|   11346008|B005EIX8JS|     125753094|LawnGarden| 2015-08-31|
|R118NNIQ75XPGO|   49206471|B000HJBKMQ|     834273114|LawnGarden| 2015-08-31|
|R30HYXHZQ49621|   37596267|B004LY59V6|     612086079|LawnGarden

In [10]:
vine_df = lawngarden_df.select('review_id','star_rating','helpful_votes','total_votes','vine')
vine_df.show()

 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RED72VWWCOS7S|          1|            2|          8|   N|
| RZHWQ208LTEPV|          5|            0|          0|   N|
|R37LBC3XAVLYOO|          5|            4|          5|   N|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|
|R22C8FMBSTFRY8|          5|            2|          2|   N|
|R118NNIQ75XPGO|          3|            0|          0|   N|
|R30HYXHZQ49621|          2|            0|          0|   N|
|R3EMLKY0GF1E90|          5|            0|          0|   N|
|R23BX7EGJMGQJR|          5|            1|          2|   N|
|R2Z4B6SDEAZF6E|          5|            0|          0|   N|
|R35289PGJERP5J|          5|            

## Write DataFrame to RDS

In [11]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.chwtrhvyl69j.us-east-2.rds.amazonaws.com/BigData"
config = {"user":"root", 
          "password": "Password", 
          "driver":"org.postgresql.Driver"}



In [12]:
customer_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [13]:
product_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [14]:
review_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [15]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)